### Building and Evaluating ARMA Models

The main task here is to pick a time series of interest, fit an ARMA model to it, make a basic forecast, and then assess
how well it performs by analyzing the errors. It’s also a good idea to experiment with a few different combinations of
$p$ and $q$, since ACF and PACF plots are only rough guides. Running a small grid search often helps find better parameter values.

For this exercise, I worked with the [Production and sales of cement bags](https://indicadores.pr/dataset/f1b1d002-5958-431a-abfe-539ce3814018/resource/806d9de9-c2ee-4506-b6fc-ed2990a54561/download/datos_cemento.csv)
, which contains monthly data on the production and sales of 94 lb sacks of cement in Puerto Rico.


## Production and sales of cement bags
#### Data Description:
The data contains three columns: `Date`, `Production`, and `Sales`. For this exercise, I will focus on forecasting the `Sales` column, which represents the number of cement bags sold each month. The goal is to build an ARMA, ARIMA, SARIMA, or SARIMAX model to forecast future sales and evaluate the model's performance.

The `Production` recording started in January 2007  and the `Sales` recording started in July 1998. For this exercise, I will work from data starting in January 2007, which gives us a consistent time frame for both production and sales.

**Source:** Government Development Bank (BGF).

Since 1959, the BGF has collected monthly information on the production and sales of companies that produce 94‑pound cement bags in Puerto Rico. At present, these are: CEMEX, ESSROC, and Argos. This information is aggregated and published by the BGF. Note: Sales of cement bags that occur in retail stores are not included, nor are cement bags imported into Puerto Rico included.


## Time Series Models

The first step is to load the data and preprocess it, including handling any missing values and ensuring that the date column is in the correct format for time series analysis. After that, I will explore the data, visualize it, and then proceed to fit different models to the sales data. Finally, I will evaluate the model's performance using appropriate metrics such as mean squared error (MSE) and mean absolute error (MAE).

To summarize, the steps are as follows:
1. Load and preprocess the data.
2. Explore and visualize the sales data.
3. Fit multiple models to the sales data.
4. Make forecasts of Sales using the fitted model.
5. Evaluate the model's performance using MSE and MAE.

In [2]:
# Import Libraries

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from openpyxl.styles.builtins import title

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Suppress SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [18]:
# Load data train data
df = pd.read_csv('./data/datos_cemento.csv')
df.head()


,Date,Production,Sales
0,12/1/2025,673242.0,1175442.649
1,11/1/2025,731303.0,1176428.391
2,10/1/2025,779872.0,1386078.940
3,9/1/2025,634802.0,1229262.336
4,8/1/2025,812366.0,1324978.434


### Preprocessing

In [19]:
# production started in Jan 2007.
df.isna().sum()

Date            0
Production    102
Sales           0
dtype: int64

In [20]:
# Drop na values
df.dropna(inplace=True)

In [21]:
df.isna().sum()

Date          0
Production    0
Sales         0
dtype: int64

In [22]:
# Should have data starting in Jan 2007
print(df.head())
print(df.tail())

        Date  Production        Sales
0  12/1/2025    673242.0  1175442.649
1  11/1/2025    731303.0  1176428.391
2  10/1/2025    779872.0  1386078.940
3   9/1/2025    634802.0  1229262.336
4   8/1/2025    812366.0  1324978.434
         Date  Production      Sales
223  5/1/2007   2912950.0  3535506.0
224  4/1/2007   2438980.0  2874418.0
225  3/1/2007   3242360.0  3530070.0
226  2/1/2007   2810360.0  3103919.0
227  1/1/2007   2338910.0  2896660.0


In [23]:
# Change date column to datetime and update index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df

,Production,Sales
Date,,
2025-12-01,673242.0,1175442.649
2025-11-01,731303.0,1176428.391
2025-10-01,779872.0,1386078.940
2025-09-01,634802.0,1229262.336
2025-08-01,812366.0,1324978.434
...,...,...
2007-05-01,2912950.0,3535506.000
2007-04-01,2438980.0,2874418.000
2007-03-01,3242360.0,3530070.000


### Exploratory Data Analysis

In [77]:
# Function to plot sales
def sales_plot(new_df=None, column='Sales', title='Monthly Sales of Cement Bags in Puerto Rico',
               xaxis_title='Date', yaxis_title='Number of Cement Bags Sold'):
    fig = go.Figure()

    # plot new df
    if new_df is not None:
        fig.add_trace(go.Scatter(
            x=new_df.index,
            y=new_df[column],
            mode='lines+markers',
            line=dict(color='#EF553B'),
            marker=dict(color='#EF553B'),
            name=column))

    # plot original Sales column
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['Sales'],
        mode='lines+markers',
        name='Sales',
        line=dict(color='#636EFA'),
        marker=dict(color='#636EFA')))

    # plot config
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        template='plotly_white',
        hovermode='x'
    )

    fig.show()


In [78]:
sales_plot(None)

#### Interpretation
This series shows trends and seasonality indicating non stationary, may need to consider differencing or seasonal components in our model. Will do stationarity test using the Augmented Dickey-Fuller test (ADF test) and also look at the ACF and PACF plots to determine potential values for p and q.

### Stationarity Test (ADF Test)

In [49]:
adf_stat, p_value, *_ = adfuller(df['Sales'])
print(f'ADF Statistic: {adf_stat}')
print(f'p-value: {p_value}')

ADF Statistic: 0.7669555902340646
p-value: 0.9910836040794191


#### Interpretation
A high p value suggest failed to reject the null hypothesis. The p value is 0.991, p > 0.05, the series is not stationary. I need to difference the series to make it stationary before fitting an ARMA model. I will apply first order differencing and then re test for stationarity.

### Differencing the series

In [50]:
df['sales_diff'] = df['Sales'].diff()
df.dropna(inplace=True)
df

,Production,Sales,sales_diff
Date,,,
2025-11-01,731303.0,1176428.391,985.742
2025-10-01,779872.0,1386078.940,209650.549
2025-09-01,634802.0,1229262.336,-156816.604
2025-08-01,812366.0,1324978.434,95716.098
2025-07-01,817408.0,1383706.313,58727.879
...,...,...,...
2007-05-01,2912950.0,3535506.000,242648.000
2007-04-01,2438980.0,2874418.000,-661088.000
2007-03-01,3242360.0,3530070.000,655652.000


In [79]:
sales_plot(df, column='sales_diff', title='Differenced Sales of Cement Bags in Puerto Rico',
           yaxis_title='Difference in Number of Cement Bags Sold')

#### Stationarity Test on Differenced Series

In [81]:
adf_stat, p_value, *_ = adfuller(df['sales_diff'])
print(f'ADF Statistic for differenced data: {adf_stat}')
print(f'p-value for differenced data: {p_value}')

ADF Statistic for differenced data: -3.194195746562607
p-value for differenced data: 0.020323120068813753


#### Interpretation
The p value is 0.02, p < 0.05, the differenced series is stationary. Now I can proceed to look at the ACF and PACF plots to determine potential values for p and q for the ARMA model.